In [1]:
import pandas as pd

In [2]:
full_results = pd.read_csv('../../cc_election_cleaning/election_results_with_vote_sponsor_cluster_FEB13.csv')

In [3]:
dldc = pd.read_csv('../../cc_election_cleaning/district_level_demo_clusters.csv')

In [11]:
dldc.sort_values(by='demo_cluster')

,district,candidate,demo_cluster
33,34,Jennifer Gutierrez,0
35,36,Chi A. Osse,0
36,37,Sandy Nurse,0
6,7,Shaun Abreu,0
7,8,Diana I. Ayala,0
8,9,Kristin Richardson Jordan,0
9,10,Carmen N. De La Rosa,0
10,11,Eric Dinowitz,0
17,18,Amanda Farias,0
16,17,Rafael Salamanca Jr.,0


In [4]:
full_results_winner = full_results[full_results['winner'] == True]

In [5]:
full_results_winner.columns

Index(['Precinct', 'vote', 'total_vote_precinct', 'vote_share', 'ed_name',
       'winner', 'rank', 'district', 'candidate', 'matched_name',
       'kmode_cluster', 'member', 'ElectDist_x', 'ML4', 'ML6', 'ML5', 'ML2',
       'ML7', 'ML3', 'ML1', 'district_cluster', 'ElectDist_y', 'Shape_Leng',
       'Shape_Area', 'geometry', 'min_noise', 'max_noise',
       'summer_noise_complaints', 'min_arrests', 'max_arrests', 'density',
       'ldensity', 'p21own', 'p21rent', 'mhhi21', 'mean_noise', 'mean_arrests',
       'living_alone_ratio', 'same_sex_ratio', 'black_nys_ratio',
       'black_fb_ratio', 'white_transplant_ratio', 'femHHH_ratio',
       'drive_ratio', 'train_ratio', 'walk_ratio', 'bike_ratio', 'wfh_ratio',
       'bus_ratio', 'hh21olongp', 'h21p', 'nha21p', 'nhb21p', 'nhw21p',
       'mex21p', 'prican21p', 'domin21p', 'colomb21p', 'kor21p', 'chin21p',
       'venez21p', 'hh21op', 'hh21rc', 'range_of_arrests', 'range_of_noise',
       'cvap21bapp', 'rv21irp', 'rv21italp', 'pg20jrbp'

In [6]:
key_demo_cols = [ 'district', 'cvap21bapp', 'mhhi21','h21p', 'nha21p', 'nhb21p', 'nhw21p','winda21p', 'white_transplant_ratio']
full__results_key_demo = full_results_winner[key_demo_cols]
ag = full__results_key_demo.groupby('district').agg('mean').reset_index()

In [7]:
demographic_cols = ['cvap21bapp', 'mhhi21', 'h21p', 'nha21p', 'nhb21p', 'nhw21p','winda21p', 'white_transplant_ratio']

In [8]:
for col in demographic_cols:
    ag[col + '_quintile'] = pd.qcut(ag[col], 5, labels=False) + 1

In [12]:
ag[ag['district'] == 38]

,district,cvap21bapp,mhhi21,h21p,nha21p,nhb21p,nhw21p,winda21p,white_transplant_ratio,cvap21bapp_quintile,mhhi21_quintile,h21p_quintile,nha21p_quintile,nhb21p_quintile,nhw21p_quintile,winda21p_quintile,white_transplant_ratio_quintile
37,38,36.109126,32537.162264,34.649075,32.466191,4.139582,25.284843,0.558541,7.473826,3,3,4,5,2,3,2,4
